In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format
# pd.options.display.int_format ='{:,}'.format

import os
from operator import itemgetter    
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')


In [2]:
import_csv_file = '../_data/operations_fulldata_nodup.csv'
df = pd.read_csv(import_csv_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128031 entries, 0 to 128030
Data columns (total 58 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   op_id              128031 non-null  int64  
 1   subject_id         128031 non-null  int64  
 2   hadm_id            128031 non-null  int64  
 3   opdate             128031 non-null  int64  
 4   age                128031 non-null  int64  
 5   sex                128031 non-null  object 
 6   weight             126611 non-null  float64
 7   height             127269 non-null  float64
 8   race               128031 non-null  object 
 9   asa                124636 non-null  float64
 10  emop               128031 non-null  int64  
 11  department         128031 non-null  object 
 12  antype             128031 non-null  object 
 13  icd10_pcs          128031 non-null  object 
 14  category_desc      128031 non-null  object 
 15  desc_short         128031 non-null  object 
 16  ca

In [3]:
# calculate teh operation durations and add to DF
df['op_duration']=df['orout_time']-df['orin_time']
df['LOS']=(df['discharge_time']-df['orout_time'])/1440 # LOS is time following surgery to dishcarge converted to days. 
operations_info = df.loc[:,['op_id','category_id','category_desc','desc_short', 'icd10_pcs', 'orin_time','orout_time', 'op_duration', 'LOS']]

operations_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128031 entries, 0 to 128030
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   op_id          128031 non-null  int64  
 1   category_id    128031 non-null  object 
 2   category_desc  128031 non-null  object 
 3   desc_short     128031 non-null  object 
 4   icd10_pcs      128031 non-null  object 
 5   orin_time      128031 non-null  int64  
 6   orout_time     128031 non-null  int64  
 7   op_duration    128031 non-null  int64  
 8   LOS            128031 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 8.8+ MB


In [4]:
import pandas as pd

TOP_VAR_PROCEDURES = 100

# Group the DataFrame by 'icd10_pcs' and 'desc_schort' and calculate descriptive statistics for 'LOS'
descriptive_stats = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].describe()

# use the agg() method with a custom list of percentiles
percentiles = [0.1, 0.9]  # 
percentile_stats = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].quantile(percentiles).unstack()

# # Concatenate the percentile stats to the descriptive stats
descriptive_stats = pd.concat([descriptive_stats, percentile_stats], axis=1)

# # Rename the percentile columns for clarity
descriptive_stats = descriptive_stats.rename(columns={0.1: '10%', 0.9: '90%'})

# MODE or additional quantiles if needed
# descriptive_stats['mode'] = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].agg(pd.Series.mode)

operations_desc_stats_df = descriptive_stats.sort_values(by='std', ascending=False).head(TOP_VAR_PROCEDURES)
# descriptive_stats.to_csv('los.csv')

operations_desc_stats_df

,,count,mean,std,min,25%,50%,75%,max,10%,90%
icd10_pcs,desc_short,,,,,,,,,,
0B110,Bypass Trachea to Esophagus with Intralum Dev,335,68,253,0,11,22,55,"3,711",5,100
00900,Drainage of Brain with Drainage Device,263,41,252,1,3,11,27,"3,728",2,56
001U0,Bypass Spinal Canal to Urinary Tract w Nonaut Sub,33,60,199,3,8,14,31,"1,157",6,65
02QA0,Repair Heart,85,31,168,1,6,9,14,"1,553",3,22
00P60,Removal of Other Device from Cereb Vent,8,79,161,8,12,17,40,475,8,183
...,...,...,...,...,...,...,...,...,...,...,...
0QH20,Insertion of Int Fix into R Pelvic Bone,3,37,22,11,30,48,50,52,19,51
0PHD3,Insertion of Int Fix into L Humeral Head,23,13,22,1,3,5,9,102,1,15
04CY0,Extirpation of Matter from Lower Artery,35,20,22,1,9,15,22,109,5,31


In [5]:
## Clean up the index. 

operations_desc_stats_df.reset_index(inplace=True)
operations_desc_stats_df.index
operations_desc_stats_df


,icd10_pcs,desc_short,count,mean,std,min,25%,50%,75%,max,10%,90%
0,0B110,Bypass Trachea to Esophagus with Intralum Dev,335,68,253,0,11,22,55,"3,711",5,100
1,00900,Drainage of Brain with Drainage Device,263,41,252,1,3,11,27,"3,728",2,56
2,001U0,Bypass Spinal Canal to Urinary Tract w Nonaut Sub,33,60,199,3,8,14,31,"1,157",6,65
3,02QA0,Repair Heart,85,31,168,1,6,9,14,"1,553",3,22
4,00P60,Removal of Other Device from Cereb Vent,8,79,161,8,12,17,40,475,8,183
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0QH20,Insertion of Int Fix into R Pelvic Bone,3,37,22,11,30,48,50,52,19,51
96,0PHD3,Insertion of Int Fix into L Humeral Head,23,13,22,1,3,5,9,102,1,15
97,04CY0,Extirpation of Matter from Lower Artery,35,20,22,1,9,15,22,109,5,31
98,07TP0,Resection of Spleen,19,16,22,3,5,7,15,94,5,31


In [6]:
## Export List of PCD codes with greatest STD DEV to CSV

# EXPORT and Specify the file path 
output_csv_file = '../_data/pcd_STD.csv'

# Save the DataFrame to a CSV file
operations_desc_stats_df.to_csv(output_csv_file, index=False)

In [14]:
## Create a list of the surgeries with the greatest Variances in Length of stay
std_devs = operations_desc_stats_df['icd10_pcs']
print(std_devs)

# # Now, filter the DataFrame to these top 100 codes
top_LOSvariance_df = operations_info[operations_info['icd10_pcs'].isin(std_devs)]

top_LOSvariance_df



0    0B110
1    00900
2    001U0
3    02QA0
4    00P60
5    0W304
6    00160
7    0BQ10
8    09Q78
9    09R70
Name: icd10_pcs, dtype: object


,op_id,category_id,category_desc,desc_short,icd10_pcs,orin_time,orout_time,op_duration,LOS
72,454625704,001,Central Nervous System and Cranial Nerves,Bypass Cereb Vent to Nasophar with Autol Sub,00160,16605,16840,235,6
216,494888235,0B1,Respiratory System,Bypass Trachea to Esophagus with Intralum Dev,0B110,44660,44750,90,16
219,431660450,001,Central Nervous System and Cranial Nerves,Bypass Cereb Vent to Nasophar with Autol Sub,00160,4800,4970,170,7
221,492893427,001,Central Nervous System and Cranial Nerves,Bypass Cereb Vent to Nasophar with Autol Sub,00160,2105,2275,170,6
590,470206316,001,Central Nervous System and Cranial Nerves,Bypass Cereb Vent to Nasophar with Autol Sub,00160,54520,54670,150,7
...,...,...,...,...,...,...,...,...,...
127823,443226485,0B1,Respiratory System,Bypass Trachea to Esophagus with Intralum Dev,0B110,5375,5480,105,7
127870,489661812,0B1,Respiratory System,Bypass Trachea to Esophagus with Intralum Dev,0B110,35495,35570,75,7
127917,458081782,0B1,Respiratory System,Bypass Trachea to Esophagus with Intralum Dev,0B110,23955,24030,75,113
127975,486837329,0B1,Respiratory System,Bypass Trachea to Esophagus with Intralum Dev,0B110,106225,106305,80,106


## Visualize LOS Distributions for surgeries

In [ ]:
import plotly.graph_objects as go

import pandas as pd

df = top_LOSvariance_df

fig = go.Figure()

pcs_list = top_LOSvariance_df['icd10_pcs']

for pcs in pcs_list:
    fig.add_trace(go.Violin(x=df['icd10_pcs'][df['icd10_pcs'] == pcs],
                            y=df['LOS'][df['icd10_pcs'] == pcs],
                            name=pcs,
                            box_visible=True,
                            meanline_visible=True))

fig.show()